### RankNet-makedb-laptime

build laptime dataset from raw log files

base: 18./laptime_rank_timediff_dataset

create laptime dataset only, no gluonts.

+ add lap2nextpit as target, this is as the same as a stacked lstm model when set prediction_length=1
+ add lap_instint to dynamic features(pitage)
+ add support for historical data, indy500 for different years
+ add track and lap status as target, preparing for StatusModel with deepAR

this notebook focuses on create the training set, while laptime2rank-evaluate will have testset codes specified for rank calculation

+ add new features, caution_laps_instint,lap_instint

Build a time series dataset across all the oval races, including laptime, rank. When this dataset aims to be used in forecasting, covariates of the racing status can not be included, such as track_status and lap_status. However, they can be used in oracle test to tell the upper bound of performance of the predictor.

Change to a new dataset format that following telemetry dataset.

raw:
+  [(eventid, carids: carno -> rowid, datalist)]

datalist := [datalist_entry] in shape of #car_number

datalist_entry := [[laptime, rank, track_status, lap_status]], in shape of #totallaps x #featureCnt (padded by nan)

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import random
import mxnet as mx
from mxnet import gluon
import pickle
import json
from gluonts.dataset.common import ListDataset
from gluonts.dataset.util import to_pandas
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

## Load Data

In [2]:
import os
os.getcwd()

'/scratch_hdd/hpda/indycar/notebook/19.RankNet'

In [3]:
# make indy car completed_laps dataset
# car_number, completed_laps, rank, elapsed_time, rank_diff, elapsed_time_diff 
def make_cl_data(dataset):

    # pick up data with valid rank
    rankdata = dataset.rename_axis('MyIdx').sort_values(by=['elapsed_time','MyIdx'], ascending=True)
    rankdata = rankdata.drop_duplicates(subset=['car_number', 'completed_laps'], keep='first')

    # resort by car_number, lap
    uni_ds = rankdata.sort_values(by=['car_number', 'completed_laps', 'elapsed_time'], ascending=True)    
    #uni_ds = uni_ds.drop(["unique_id", "best_lap", "current_status", "track_status", "lap_status",
    #                  "laps_behind_leade","laps_behind_prec","overall_rank","pit_stop_count",
    #                  "last_pitted_lap","start_position","laps_led"], axis=1)
    
    uni_ds = uni_ds.drop(["unique_id", "best_lap", 
                      "laps_behind_leade","laps_behind_prec","overall_rank","pit_stop_count",
                      "last_pitted_lap","start_position","laps_led"], axis=1)
        
    carnumber = set(uni_ds['car_number'])
    print('cars:', carnumber)
    print('#cars=', len(carnumber))
   
    # faster solution , uni_ds already sorted by car_number and lap
    uni_ds['rank_diff'] = uni_ds['rank'].diff()
    mask = uni_ds.car_number != uni_ds.car_number.shift(1)
    uni_ds['rank_diff'][mask] = 0
    
    uni_ds['time_diff'] = uni_ds['elapsed_time'].diff()
    mask = uni_ds.car_number != uni_ds.car_number.shift(1)
    uni_ds['time_diff'][mask] = 0
    
    #df = uni_ds[['car_number','completed_laps','rank','elapsed_time','rank_diff','time_diff']]
    #df = uni_ds[['car_number','completed_laps','rank',
    #             'rank_diff','time_diff',"current_status", "track_status", "lap_status",'elapsed_time']]
    
    df = uni_ds[['car_number','completed_laps','time_diff','rank','track_status', 'lap_status','elapsed_time']]
    
    return df

def make_lapstatus_data(dataset):
    final_lap = max(dataset.completed_laps)
    total_laps = final_lap + 1

    # get records for the cars that finish the race
    completed_car_numbers= dataset[dataset.completed_laps == final_lap].car_number.values
    completed_car_count = len(completed_car_numbers)

    print('count of completed cars:', completed_car_count)
    print('completed cars:', completed_car_numbers)
    
    #pick up one of them
    onecar = dataset[dataset['car_number']==completed_car_numbers[0]]
    onecar = onecar.drop_duplicates(subset=['car_number', 'completed_laps'], keep='first')
    return onecar[['completed_laps','track_status']]
    

In [4]:
def load_data(event, year=0):
    #inputfile = '../data/final/C_'+ event +'-' + year + '-final.csv'
    if year>0:
        inputfile = '../data/final/C_'+ event +'-' + year + '.csv'
    else:
        inputfile = '../data/final/C_'+ event +'.csv'
    
    #outputprefix = year +'-' + event + '-'
    dataset = pd.read_csv(inputfile)
    #dataset.info(verbose=True)    
    
    final_lap = max(dataset.completed_laps)
    total_laps = final_lap + 1

    # get records for the cars that finish the race
    completed_car_numbers= dataset[dataset.completed_laps == final_lap].car_number.values
    completed_car_count = len(completed_car_numbers)

    print('count of completed cars:', completed_car_count)
    print('completed cars:', completed_car_numbers)

    #make a copy
    alldata = dataset.copy()
    dataset = dataset[dataset['car_number'].isin(completed_car_numbers)]
    rankdata = alldata.rename_axis('MyIdx').sort_values(by=['elapsed_time','MyIdx'], ascending=True)
    rankdata = rankdata.drop_duplicates(subset=['car_number', 'completed_laps'], keep='first')
    
    cldata = make_cl_data(dataset)
    flagdata = make_lapstatus_data(dataset)
    acldata = make_cl_data(alldata)

    return alldata, rankdata, acldata, flagdata

### overall view of laptime scatter plots



In [5]:
def get_cardata(curcarno, ycol='time_diff'):
    car = acldata[acldata['car_number']==curcarno]
    #print(car['time_diff'].describe())
    
    cols=['completed_laps','rank','car_number','lap_status','track_status',
    'pit_stop_count','current_status','start_position']
    colid={key:idx for idx, key in enumerate(cols)}

    cardata = rankdata[rankdata['car_number'] == curcarno]

    carstatus = [[row[0], row[1],row[2],row[3],row[4],row[5],row[6],row[7]] for row in cardata[
        ['completed_laps','rank','car_number','lap_status','track_status',
        'pit_stop_count','current_status','start_position']].values]
    
    x = car['completed_laps'][1:].values
    y = car[ycol][1:].values

    pits=[]
    yellowflags=[]
    lastflag = 'x'
    for row in carstatus:
        lap = int(row[colid['completed_laps']])

        if row[colid['lap_status']]=='P':
            pits.append(lap)

        if row[colid['track_status']]=='Y':
            if lastflag != 'Y':       
                #start
                yellowflags.append(lap)
        else:
            if lastflag == 'Y':       
                #end
                yellowflags.append(lap)        
        lastflag = row[colid['track_status']]

    #pit lap
    pits = np.array(pits)
    #start, end lap
    #
    yellowflags = np.array(yellowflags)
    if (yellowflags.shape[0] % 2)==1:
        print('crash?:carno=', curcarno)
        yellowflags = []
    else:
        yellowflags = np.array(yellowflags).reshape((-1,2))    
    
    return car, x, y, pits, yellowflags

### bulid the dataset

In [6]:
def nan_helper(y):
    """Helper to handle indices and logical indices of NaNs.

    Input:
        - y, 1d numpy array with possible NaNs
    Output:
        - nans, logical indices of NaNs
        - index, a function, with signature indices= index(logical_indices),
          to convert logical indices of NaNs to 'equivalent' indices
    Example:
        >>> # linear interpolation of NaNs
        >>> nans, x= nan_helper(y)
        >>> y[nans]= np.interp(x(nans), x(~nans), y[~nans])
    """

    return np.isnan(y), lambda z: z.nonzero()[0]

def get_lap2nextpit(lap_status, maxlap=200):
    """
    input:
        lapstatus  ; array of 0/1 indicating pitstops for each lap, nan means incomplete race
        maxlap     ; the max lap number of the race
    output:
        lap2nextpit ; array of the lap gap to the next pit for each lap
    
    """
    
    #pitstops = np.where(lap_status==1)[0]
                    
    pitstops = list(np.where(lap_status==1)[0])
    #if not len(lap_status) < maxlap:
    nans, x= nan_helper(lap_status)
    nan_count = np.sum(nans)      
    if nan_count == 0:
        #complete cars
        # the last stint, to the end
        pitstops.append(maxlap)
    
    lap2nextpit = np.zeros_like(lap_status)
    lap2nextpit[:] = np.nan
    
    #guard
    if len(pitstops)==0:
        return lap2nextpit
    
    idx = 0
    for lap in range(len(lap_status)):
        if lap < pitstops[idx]:
            lap2nextpit[lap] = pitstops[idx] - lap
        else:
            idx += 1
            if idx < len(pitstops):
                lap2nextpit[lap] = pitstops[idx] - lap
            else:
                break
            
    return lap2nextpit


In [7]:
def get_lapdata(acldata):
    """
    input:
        acldata['car_number','completed_laps','time_diff','rank','track_status', 'lap_status','elapsed_time']
    
        timediff: [car_number, completed_laps] -> elapsed time diff to leader
    output:
        lapdata = acldata[['car_number','completed_laps',
                           'time_diff','rank','track_status', 'lap_status','time_behind']].to_numpy()
    """
    COL_COMPLETED_LAPS = 1
    COL_ELAPSED_TIME = 6
    
    maxlap = np.max(acldata['completed_laps'].values)
    #'car_number','completed_laps','time_diff','rank','track_status', 'lap_status','time_behind'
    time_behind = []
    
    for lap in range(1, maxlap+1):
        this_lap = acldata[acldata['completed_laps']==lap][
            ['car_number','completed_laps','time_diff','rank',
             'track_status', 'lap_status','elapsed_time']].values
        
        min_elapsed_time = np.nanmin(this_lap[:,COL_ELAPSED_TIME].astype(np.float))
        #print(f'lap:{lap}, min_elapsed_time:{min_elapsed_time}')
        
        for row in this_lap:
            car_number = int(row[0])
            time_diff = row[2]
            rank = row[3]
            track_status = row[4]
            lap_status = row[5]
            
            timebehind = float(row[COL_ELAPSED_TIME]) - min_elapsed_time
            #
            time_behind.append([car_number, lap, time_diff,rank,track_status, lap_status,
                                timebehind, float(row[COL_ELAPSED_TIME])])
    
    #return
    lapdata = np.array(time_behind)
    return lapdata



# features: laptime, rank, track_status, lap_status, timediff
LAPTIME = 0
RANK = 1
TRACK_STATUS = 2
LAP_STATUS = 3
TIME_BEHIND = 4
CAUTION_LAPS_INSTINT = 5 
LAPS_INSTINT = 6
ELAPSED_TIME = 7
LAP2NEXTPIT = 8

_featureCnt = 9
        
def get_laptime_dataset(stagedata, inlap_status = 0):
    """
    #add caution_laps_instint, laps_instint
    
    input: (alldata, rankdata, acldata, flagdata)
    output: laptime & rank data
    
    [(
    eventid,
    carids : rowid -> carno,
    datalist: #car_number x features x #totallaps (padded by Nan)
        entry: [[laptime, rank, track_status, lap_status,
                caution_laps_instint, laps_instint]]
    )]
    """
    laptime_data = []
    for event in stagedata.keys():
        
        print(f'start event: {event}')
        
        laptime_rec = []
        eventid = events_id[event]
        
        alldata, rankdata, acldata, flagdata = stagedata[event]
        carlist = set(acldata['car_number'])
        laplist = set(acldata['completed_laps'])
        totalcars = len(carlist)
        totallaps = len(laplist)
        


        #carnumber -> carid
        carids={key:idx for idx, key in enumerate(carlist)}
        decode_carids={idx:key for idx, key in enumerate(carlist)}

        #init
        lap_instint = {carids[x]:0 for x in carlist}
        caution_instint = {carids[x]:0 for x in carlist}        
        
        #array: car_number x lap
        #laptime = np.zeros((totalcars, totallaps-1))
        #rank = np.zeros((totalcars, totallaps-1))
        laptime = np.empty((totalcars, totallaps-1))
        rank = np.empty((totalcars, totallaps-1))
        laptime[:] = np.NaN
        rank[:] = np.NaN
        

        datalist = np.empty((totalcars, _featureCnt, totallaps-1))
        datalist[:] = np.NaN
        
        #lapdata = acldata[['car_number','completed_laps',
        #                   'time_diff','rank','track_status', 'lap_status','elapsed_time']].to_numpy()
        
        #'car_number','completed_laps','time_diff','rank','track_status', 'lap_status','time_behind'
        lapdata = get_lapdata(acldata)
        
        
        for row in lapdata:
            #completed_laps
            if int(row[1]) == 0:
                continue
                
            #add to data array
            car_number = carids[int(row[0])]
            completed_laps = int(row[1])-1
            time_diff = float(row[2])
            rank = int(row[3])
            track_status = 1 if row[4]=='Y' else 0
            lap_status = 1 if row[5]=='P' else 0
            time_behind = float(row[6])
            
            datalist[car_number, LAPTIME, completed_laps] = time_diff
            datalist[car_number, RANK, completed_laps] = rank
            datalist[car_number, TRACK_STATUS, completed_laps] = track_status
            datalist[car_number, LAP_STATUS, completed_laps] = lap_status
            datalist[car_number, TIME_BEHIND, completed_laps] = time_behind

            datalist[car_number, ELAPSED_TIME, completed_laps] = float(row[7])

            
            #stint status
            if track_status == 1:
                caution_instint[car_number] += 1
            lap_instint[car_number] += 1
            if lap_status == 1:
                #new stint
                lap_instint[car_number] = 0
                caution_instint[car_number] = 0
                
                # add inlap feature into lap_Status
                # set the previous lap to inlap status
                
                # what does it mean?
                
                if (inlap_status!=0):
                    if inlap_status == 1:
                        # set the previous lap of 'P'
                        if completed_laps > 0:
                            #datalist[car_number, LAP_STATUS, completed_laps-1] = INLAP_STATUS
                            datalist[car_number, LAP_STATUS, completed_laps-1] = 1
                    else:
                        # set the next lap of 'P'
                        if completed_laps +1 < totallaps:
                            #datalist[car_number, LAP_STATUS, completed_laps-1] = INLAP_STATUS
                            datalist[car_number, LAP_STATUS, completed_laps + 1] = 1
                
            
            datalist[car_number, LAPS_INSTINT, completed_laps] = lap_instint[car_number]
            datalist[car_number, CAUTION_LAPS_INSTINT, completed_laps] = caution_instint[car_number]
                

                
        #update lap2nextpit in datalist
        for caridx in range(datalist.shape[0]):
            lap_status = datalist[caridx, LAP_STATUS, :]
            #pit status
            lap2nextpit = get_lap2nextpit(lap_status)
            datalist[caridx, LAP2NEXTPIT, :] = lap2nextpit        
                
        #add one record
        laptime_data.append([eventid, decode_carids, datalist])
        # push this event into stage dataframe
        print('event=%s, records=%s'%(event, datalist.shape))
        
    
    return laptime_data

### load data

In [8]:
stagedata = {}
global_carids = {}
traindata = None
cur_carid = 0

#inlap status = 
# 0 , no inlap
# 1 , set previous lap
# 2 , set the next lap
_inlap_status = 2

years = ['2013','2014','2015','2016','2017','2018','2019']
#years = ['2013','2014','2015','2016','2017','2018']
#events = ['Indy500']
events = [f'Indy500-{x}' for x in years]
events_id={key:idx for idx, key in enumerate(events)}

dbid = f'Indy500_{years[0]}_{years[-1]}_v{_featureCnt}_p{_inlap_status}'

for event in events:
    #dataid = f'{event}-{year}'
    #alldata, rankdata, acldata, flagdata
    stagedata[event] = load_data(event)

    alldata, rankdata, acldata, flagdata = stagedata[event]
    carlist = set(acldata['car_number'])
    laplist = set(acldata['completed_laps'])
    print('%s: carno=%d, lapnum=%d'%(event, len(carlist), len(laplist)))

    #build the carid map
    for car in carlist:
        if car not in global_carids:
            global_carids[car] = cur_carid
            cur_carid += 1
    


count of completed cars: 19
completed cars: [11 26  1 25 19  3  2 77 83 20 22  8 14  9 18 55 78  5 12]
cars: {1, 2, 3, 5, 8, 9, 11, 12, 77, 14, 78, 18, 19, 20, 83, 22, 55, 25, 26}
#cars= 19
count of completed cars: 19
completed cars: [11 26  1 25 19  3  2 77 83 20 22  8 14  9 18 55 78  5 12]
cars: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 18, 19, 20, 21, 22, 25, 26, 27, 41, 55, 60, 63, 77, 78, 81, 83, 91, 98}
#cars= 33
Indy500-2013: carno=33, lapnum=201
count of completed cars: 20
completed cars: [28  3 25 34  2 26 11 12 22 21 16 77 68  5 17 33 18  8 14 98]
cars: {33, 2, 3, 34, 5, 68, 98, 8, 11, 12, 77, 14, 16, 17, 18, 21, 22, 25, 26, 28}
#cars= 20
count of completed cars: 20
completed cars: [28  3 25 34  2 26 11 12 22 21 16 77 68  5 17 33 18  8 14 98]


/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


cars: {2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 25, 26, 27, 28, 33, 34, 41, 63, 67, 68, 77, 83, 91, 98}
#cars= 33
Indy500-2014: carno=33, lapnum=201
count of completed cars: 20
completed cars: [ 2  1 83  9 15 27  3  6 21 22 11  5 14 24 28 98 48  7 29 26]
cars: {1, 2, 3, 98, 5, 6, 7, 9, 11, 14, 15, 48, 83, 21, 22, 24, 26, 27, 28, 29}
#cars= 20
count of completed cars: 20
completed cars: [ 2  1 83  9 15 27  3  6 21 22 11  5 14 24 28 98 48  7 29 26]
cars: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 14, 15, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 32, 41, 43, 48, 63, 83, 88, 98}
#cars= 33
Indy500-2015: carno=33, lapnum=201
count of completed cars: 17
completed cars: [98 26 21 10 42  6  5  9 11 12  3 77 27 15  8 41 35]
cars: {98, 3, 35, 5, 6, 8, 9, 10, 11, 12, 41, 42, 15, 77, 21, 26, 27}
#cars= 17
count of completed cars: 17
completed cars: [98 26 21 10 42  6  5  9 11 12  3 77 27 15  8 41 35]
cars: {2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 18, 19, 20, 21, 2

In [9]:
laptime_data = get_laptime_dataset(stagedata,inlap_status = _inlap_status)

start event: Indy500-2013
event=Indy500-2013, records=(33, 9, 200)
start event: Indy500-2014
event=Indy500-2014, records=(33, 9, 200)
start event: Indy500-2015
event=Indy500-2015, records=(33, 9, 200)
start event: Indy500-2016
event=Indy500-2016, records=(33, 9, 200)
start event: Indy500-2017
event=Indy500-2017, records=(33, 9, 200)
start event: Indy500-2018
event=Indy500-2018, records=(33, 9, 200)
start event: Indy500-2019
event=Indy500-2019, records=(33, 9, 200)


In [10]:
import pickle
#stintdf.to_csv('laptime-%s.csv'%year)
savefile = f'laptime_rank_timediff_pit-oracle-{dbid}.pickle' 
print(savefile)
with open(savefile, 'wb') as f:
    #pack [global_carids, laptime_data]
    savedata = [global_carids, laptime_data]
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(savedata, f, pickle.HIGHEST_PROTOCOL)

laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p2.pickle


### test

In [11]:
sys.exit(0)

SystemExit: 0

/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
lapstatusA = laptime_data[-1][2][1,3,:]
#lapstatusA = laptime_data[-1][2][0,3,:]
lapstatusA

In [ ]:
lap2nextpit = laptime_data[-1][2][1,8,:]
#lap2nextpit = laptime_data[-1][2][0,8,:]
lap2nextpit

In [ ]:
#with open('laptime_rank_timediff_fulltest-oracle-%s.pickle'%year, 'rb') as f:
with open(f'laptime_rank_timediff_pit-oracle-{dbid}.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    global_carids, laptime_data = pickle.load(f, encoding='latin1') 

In [ ]:
def test_a(a, bitflag):
    return (a & bitflag) ==  bitflag

test_a(5, 4)


In [ ]:
mat = np.random.randint(5, size=(10,2, 5)).astype(np.float)
mat[2,1,0] = np.nan
_idx = np.array([0,1,2,8])
selmat = mat[_idx]
selmat[:,1,:]

In [ ]:
mask = np.isnan(selmat[:,1,:])
mask

In [ ]:
true_rank.shape

In [ ]:
idx = np.argsort(selmat[:,1,:], axis=0)
true_rank = np.argsort(idx, axis=0).astype(np.float)
true_rank[mask] = np.nan
true_rank

In [ ]:
selmat[:,1,:] = true_rank
selmat

In [ ]:
mat[_idx]

In [ ]:
mat[_idx] = selmat

In [ ]:
mat[_idx]

In [ ]:
freq, prediction_length, cardinality = 0,0,0

def load_dataset(inputfile):
    global freq, prediction_length, cardinality

    with open(inputfile, 'rb') as f:
        # have to specify it.
        freq, prediction_length, cardinality,train_ds, test_ds = pickle.load(f, encoding='latin1')
    
    print(f"number of cars: {cardinality}")
    
    return train_ds, test_ds

train_ds, test_ds = load_dataset('rank-oracle-noip-eid-all-all-f1min-t30-rIndy500-gluonts-indy-2018.pickle')

In [ ]:
ds_iter =  iter(test_ds)
rec = next(ds_iter)
rec

In [ ]:
def debug_get_ranklist(train_ds):
    ds_iter =  iter(train_ds)
    ranklist = []
    for idx in range(len(train_ds)):
        rec = next(ds_iter)

        if rec['feat_static_cat'][1] == 0:
            ranklist.append(rec['target'][0])

    return np.array(sorted(ranklist))

In [ ]:
len(ranklist)

In [ ]:
rank_train, rank_ds = get_testds(True)

In [ ]:
rawrank_train, rawrank_ds = get_testds(False)

In [ ]:
debug_get_ranklist(rank_train)

In [ ]:
debug_get_ranklist(rawrank_train)

In [ ]:
debug_get_ranklist(train_ds)

In [ ]:
 #alldata, rankdata, acldata, flagdata = stagedata['Indy500-2011']

In [ ]:
set(acldata.car_number.values)

In [ ]:
acldata

In [ ]:
acldata[acldata['completed_laps']==187]

In [ ]:
print(','.join(list(sorted(rankdata[rankdata['completed_laps']==186].car_number.values))))
print(','.join(list(sorted(rankdata[rankdata['completed_laps']==187].car_number.values))))
print(','.join(list(sorted(rankdata[rankdata['completed_laps']==188].car_number.values))))
print(','.join(list(sorted(rankdata[rankdata['completed_laps']==189].car_number.values))))


In [ ]:
alldata, rankdata, acldata, flagdata = stagedata['Indy500-2014']

In [ ]:
kannan = rankdata[rankdata['car_number']==10]
kannan[(kannan['completed_laps'] > 60) & (kannan['completed_laps']<70)]

In [ ]:
alldata, rankdata, acldata, flagdata = stagedata['Indy500-2013']
rankdata[(rankdata['car_number']==16) & (rankdata['completed_laps']<10)]

In [ ]:
rankdata[(rankdata['car_number']==16) & (rankdata['completed_laps']>88)& (rankdata['completed_laps']<97)]

In [ ]:
car16 = acldata[acldata['car_number']==16]


In [ ]:
for idx in range(len(car16)):
    if(car16.iloc[idx].lap_status == 'P'):
        print(car16.iloc[idx].completed_laps)

In [ ]:
for idx in range(len(car16)):
    if(car16.iloc[idx].track_status == 'Y'):
        print(car16.iloc[idx].completed_laps)

In [ ]:
### check the inlap
alldata, rankdata, acldata, flagdata = stagedata['Indy500-2018']
car12 = acldata[acldata['car_number']==12]
for idx in range(len(car12)):
    if(car12.iloc[idx].lap_status == 'P'):
        print(car12.iloc[idx].completed_laps)

In [ ]:
pitlap = 32
rankdata[(rankdata['car_number']==12) & (rankdata['completed_laps']>=pitlap -2) & ((rankdata['completed_laps']<=pitlap+2))]

In [ ]:
pitlap = 50
rankdata[(rankdata['car_number']==12) & (rankdata['completed_laps']>=pitlap -2) & ((rankdata['completed_laps']<=pitlap+2))]

In [ ]:
pitlap = 94
rankdata[(rankdata['car_number']==12) & (rankdata['completed_laps']>=pitlap -2) & ((rankdata['completed_laps']<=pitlap+2))]